# Working with the TAPE `Ensemble` object

When working with many lightcurves, the TAPE `Ensemble` object serves as a singular interface for storing, filtering, and analyzing timeseries data. 
Let's consider an example set of lightcurves, generated as follows:

In [1]:
import numpy as np
np.random.seed(1)

# initialize a dictionary of empty arrays
source_dict = {"id": np.array([]),
                   "time": np.array([]),
                   "flux": np.array([]),
                   "error": np.array([]),
                   "band": np.array([])}

# Create 10 lightcurves with 100 measurements each
lc_len = 100
for i in range(10):
    source_dict["id"] = np.append(source_dict["id"], np.array([i]*lc_len)).astype(int)
    source_dict["time"] = np.append(source_dict["time"], np.linspace(1, lc_len, lc_len))
    source_dict["flux"] = np.append(source_dict["flux"], 100 + 50 * np.random.rand(lc_len))
    source_dict["error"] = np.append(source_dict["error"], 10 + 5 * np.random.rand(lc_len))
    source_dict["band"] = np.append(source_dict["band"], ["g"]*50+["r"]*50)

We can load these into the `Ensemble` using `Ensemble.from_source_dict()`:

In [2]:
from tape.ensemble import Ensemble

ens = Ensemble()  # initialize an ensemble object

# Read in the generated lightcurve data
ens.from_source_dict(source_dict, 
                     id_col="id",
                     time_col="time",
                     flux_col="flux",
                     err_col="error",
                     band_col="band")

We now have an `Ensemble` object, and have provided it with the constructed data in the source dictionary. Within the call to `Ensemble.from_source_dict`, we specified which columns of the input file mapped to timeseries quantities that the `Ensemble` needs to understand. It's important to link these arguments properly, as the `Ensemble` will use these columns when operations are requested on understood quantities. For example, if an TAPE analysis function requires the time column, from this linking the `Ensemble` will automatically supply that function with the 'time' column.

## Column Mapping with the ColumnMapper

In the above example, we manually provide the column labels within the call to `Ensemble.from_source_dict`. Alternatively, the `tape.utils.ColumnMapper` class offers a means to assign the column mappings. Either manually as shown before, or even populated from a known mapping scheme.

In [3]:
from tape.utils import ColumnMapper

# columns assigned manually
col_map = ColumnMapper().assign(id_col="id",
                                time_col="time",
                                flux_col="flux",
                                err_col="error",
                                band_col="band")

# Pass the ColumnMapper along to from_source_dict
ens.from_source_dict(source_dict, column_mapper=col_map)

## The Object and Source Frames
The `Ensemble` maintains two dataframes under the hood, the "object dataframe" and the "source dataframe". This borrows from the Rubin Observatories object-source convention, where object denotes a given astronomical object and source is the collection of measurements of that object. Essentially, the Object frame stores one-off information about objects, and the source frame stores the available time-domain data. As a result, `Ensemble` functions that operate on the underlying dataframes need to be pointed at either object or source. In most cases, the default is the object table as it's a more helpful interface for understanding the contents of the `Ensemble`, especially when dealing with large volumes of data.

## Dask and "Lazy Evaluation"

Before going any further, the `Ensemble` is built on top of `Dask`, which brings with it a powerful framework for parallelization and scalability. However, there are some differences in how `Dask` code works that, if you're unfamiliar with it, is worth establishing right here at the get-go. The first is that `Dask` evaluates code "lazily". Meaning that many operations are not executed when the line of code is run, but instead are added to a scheduler to be executed when the result is actually needed. See below for an example:

In [4]:
ens._source  # We have not actually loaded any data into memory

,time,flux,error,band
npartitions=1,,,,
0,float64,float64,float64,string
9,...,...,...,...


Here we are accessing the Dask dataframe underneath, and despite running a command to read in our data, we only see an empty dataframe with some high-level information available. To explicitly bring the data into memory, we must run a `compute()` command.

In [5]:
ens.compute("source")  # Compute lets dask know we're ready to bring the data into memory

,time,flux,error,band
id,,,,
0,1.0,120.851100,11.633225,g
0,2.0,136.016225,12.635291,g
0,3.0,100.005719,14.429710,g
0,4.0,115.116629,11.786349,g
0,5.0,107.337795,14.542676,g
...,...,...,...,...
9,96.0,138.371176,12.237541,r
9,97.0,104.060829,10.920638,r
9,98.0,149.920678,14.143664,r


With this compute, we see above that we now have a populated dataframe (a Pandas dataframe in fact!). From this, many workflows in Dask and by extension TAPE, will look like a series of lazily evaluated commands that are chained together and then executed with a .compute() call at the end of the workflow.

Alternatively we can use `ens.persist()` to execute the chained commands without loading the result into memory. This can speed up future `compute()` calls.

## Inspection, Filtering, and Selecting

The `Ensemble` contains an assortment of functions for inspecting and filtering your data.

### Inspection

These functions provide views into the contents of your `Ensemble` dataframe, especially important when dealing with large data volumes that cannot be brought into memory all at once. The first is `Ensemble.info` which provides information on the columns, data types, and memory usage of the dataframe.

In [6]:
# Inspection

ens.info(verbose=True, memory_usage=True)  # Grabs high level information about the dataframes

Object Table
<class 'dask.dataframe.core.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   nobs_g      10 non-null      float64
 1   nobs_r      10 non-null      float64
 2   nobs_total  10 non-null      float64
dtypes: float64(3)
memory usage: 320.0 bytes
Source Table
<class 'dask.dataframe.core.DataFrame'>
Index: 1000 entries, 0 to 9
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   time    1000 non-null      float64
 1   flux    1000 non-null      float64
 2   error   1000 non-null      float64
 3   band    1000 non-null      string
dtypes: float64(3), string(1)
memory usage: 36.1 KB


`Ensemble.info` shows that we have 2000 rows with 54.7 KBs of used memory, and shows the columns we've brought in with their respective data types. If you'd like to actually bring a few rows into memory to inspect, `Ensemble.head` and `Ensemble.tail` provide access to the first n and last n rows respectively.

In [7]:
ens.head("object", 5)  # Grabs the first 5 rows of the object table

band,nobs_g,nobs_r,nobs_total
id,,,
0,50,50,100
1,50,50,100
2,50,50,100
3,50,50,100
4,50,50,100


In [8]:
ens.tail("source", 5)  # Grabs the last 5 rows of the source table

,time,flux,error,band
id,,,,
9,96.0,138.371176,12.237541,r
9,97.0,104.060829,10.920638,r
9,98.0,149.920678,14.143664,r
9,99.0,119.480601,10.154990,r
9,100.0,145.260138,14.733641,r


Additionally, when you are working with a small enough dataset, `Ensemble.compute` can be used to bring the whole dataframe into memory (as shown previously). 

In [9]:
ens.compute("source")

,time,flux,error,band
id,,,,
0,1.0,120.851100,11.633225,g
0,2.0,136.016225,12.635291,g
0,3.0,100.005719,14.429710,g
0,4.0,115.116629,11.786349,g
0,5.0,107.337795,14.542676,g
...,...,...,...,...
9,96.0,138.371176,12.237541,r
9,97.0,104.060829,10.920638,r
9,98.0,149.920678,14.143664,r


### Filtering

The `Ensemble` provides a general filtering function `query` that mirrors a Pandas or Dask `query` command. Specifically, the function takes a string that provides an expression indicating which rows to **keep**. As with other `Ensemble` functions, an optional `table` parameter allows you to filter on either the object or the source table.

For example, the following code filters the sources to only include rows with a flux value above 18.2. It uses `ens._flux_col` to retrieve the name of the column with that information.

In [10]:
ens.query(f"{ens._flux_col} > 130.0", table="source")
ens.compute("source")

,time,flux,error,band
id,,,,
0,2.0,136.016225,12.635291,g
0,12.0,134.260975,10.685679,g
0,14.0,143.905872,13.484091,g
0,16.0,133.523376,13.777315,g
0,21.0,140.037228,10.099401,g
...,...,...,...,...
9,91.0,140.368263,14.320720,r
9,92.0,148.476901,12.239495,r
9,96.0,138.371176,12.237541,r


Alternatively, we could use a Dask dataseries of Booleans to indicate which rows to *keep*. We can often compute these series as the result of some operation on the underlying tables:

In [11]:
keep_rows = ens._source["error"] < 12.0
keep_rows.compute()

id
0    False
0     True
0    False
0    False
0     True
     ...  
9    False
9    False
9    False
9    False
9    False
Name: error, Length: 422, dtype: bool

We then pass that series to a `filter_from_series` function:

In [12]:
ens.filter_from_series(keep_rows, table="source")
ens.compute("source")

,time,flux,error,band
id,,,,
0,12.0,134.260975,10.685679,g
0,21.0,140.037228,10.099401,g
0,22.0,148.413079,10.131055,g
0,24.0,134.616131,11.231055,g
0,30.0,143.907125,11.395918,g
...,...,...,...,...
9,81.0,149.016644,10.755373,r
9,85.0,130.071670,11.960329,r
9,86.0,136.297942,11.419338,r


Additionally, several more specific functions are available for common operations.

In [13]:
# Cleaning nans
ens.dropna(table="source")  # clean nans from source table
ens.dropna(table="object")  # clean nans from object table

# Filtering on number of observations
ens.prune(threshold=10)  # threshold is the minimum number of observations needed to retain the object

ens.info(verbose=True)

Object Table
<class 'dask.dataframe.core.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   nobs_g      10 non-null      float64
 1   nobs_r      10 non-null      float64
 2   nobs_total  10 non-null      float64
dtypes: float64(3)
memory usage: 320.0 bytes
Source Table
<class 'dask.dataframe.core.DataFrame'>
Index: 169 entries, 0 to 9
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   time    169 non-null      float64
 1   flux    169 non-null      float64
 2   error   169 non-null      float64
 3   band    169 non-null      string
dtypes: float64(3), string(1)
memory usage: 6.1 KB


In the above operations, we remove any rows that have at least 1 NaN value present. And then filter such that only lightcurves which have at least 50 measurements are retained.

### Selecting

The `Ensemble` also provides a `select` function to filter down to a subset of columns.

In [14]:
# Add a new column so we can filter it out later.
ens._source = ens._source.assign(band2=ens._source["band"] + "2")
ens.compute("source")

,time,flux,error,band,band2
id,,,,,
0,12.0,134.260975,10.685679,g,g2
0,21.0,140.037228,10.099401,g,g2
0,22.0,148.413079,10.131055,g,g2
0,24.0,134.616131,11.231055,g,g2
0,30.0,143.907125,11.395918,g,g2
...,...,...,...,...,...
9,81.0,149.016644,10.755373,r,r2
9,85.0,130.071670,11.960329,r,r2
9,86.0,136.297942,11.419338,r,r2


In [15]:
ens.select(["time", "flux", "error", "band"], table="source")
ens.compute("source")

,time,flux,error,band
id,,,,
0,12.0,134.260975,10.685679,g
0,21.0,140.037228,10.099401,g
0,22.0,148.413079,10.131055,g
0,24.0,134.616131,11.231055,g
0,30.0,143.907125,11.395918,g
...,...,...,...,...
9,81.0,149.016644,10.755373,r
9,85.0,130.071670,11.960329,r
9,86.0,136.297942,11.419338,r


## Assignments and Column Manipulation

The ensemble object supports assignment through the Dask `assign` function. We can pass in either a callable or a series to assign to the new column. New column names are produced automatically from the argument name.

For example, if we want to compute the lower bound of an error range as the estimated flux minus twice the estimated error, we would use:

In [16]:
ens.assign(table="source", lower_bnd=lambda x: x["flux"] - 2.0 * x["error"])
ens.compute(table="source")

,time,flux,error,band,lower_bnd
id,,,,,
0,12.0,134.260975,10.685679,g,112.889618
0,21.0,140.037228,10.099401,g,119.838427
0,22.0,148.413079,10.131055,g,128.150969
0,24.0,134.616131,11.231055,g,112.154020
0,30.0,143.907125,11.395918,g,121.115288
...,...,...,...,...,...
9,81.0,149.016644,10.755373,r,127.505899
9,85.0,130.071670,11.960329,r,106.151012
9,86.0,136.297942,11.419338,r,113.459267


## Batch Analysis

The `Ensemble` provides a powerful batching interface, `Ensemble.batch`, with in-built parallelization (provided the input data is in multiple partitions). In addition, TAPE has a suite of analysis functions on-hand for your use. Below, we show the application of `tape.analysis.calc_stetson_J` on our dataset.

In [17]:
# using tape analysis functions
from tape.analysis import calc_stetson_J

res = ens.batch(calc_stetson_J, compute=True)  # compute is set to true to execute immediately (non-lazily)
res

id
0    {'g': -0.8833723170736909, 'r': -0.81291313232...
1    {'g': -0.7866661902102343, 'r': -0.79927945599...
2    {'g': -0.8650811883274131, 'r': -0.87939085289...
3    {'g': -0.9140015912865537, 'r': -0.90284371456...
4    {'g': -0.8232578922439672, 'r': -0.81922455220...
5    {'g': -0.668795976899231, 'r': -0.784477243304...
6    {'g': -0.8115552290707235, 'r': -0.90666227394...
7    {'g': -0.6217573153267577, 'r': -0.60999974938...
8    {'g': -0.7001359525394822, 'r': -0.73620435205...
9    {'g': -0.7266040976469818, 'r': -0.68878460237...
Name: stetsonJ, dtype: object

## Using light-curve package features

`Ensemble.batch` also supports the use of [light-curve](https://pypi.org/project/light-curve/) package feature extractor:

In [18]:
import light_curve as licu

extractor = licu.Extractor(licu.Amplitude(), licu.AndersonDarlingNormal(), licu.StetsonK())
res = ens.batch(extractor, compute=True, band_to_calc="g")
res

,amplitude,anderson_darling_normal,stetson_K
id,,,
0,7.076052,0.177751,0.834036
1,8.591493,0.513749,0.769344
2,8.141189,0.392628,0.856307
3,5.751674,0.295631,0.809191
4,7.871321,0.555775,0.849305
5,8.666473,0.342937,0.823194
6,8.649326,0.241117,0.832815
7,8.856443,1.141906,0.772267
8,9.297713,0.984247,0.968132


## Using a Custom Analysis Function
The analysis functions contained in TAPE are meant to provide a collection of performant, on-hand routines for common timeseries use cases. However, TAPE is also equipped to handle externally defined functions. Let's walk through a short example of defining a simple custom function and applying it through `Ensemble.batch`.

Here we define a simple function, that returns an average flux for each photometric band. It requires an array of fluxes, an array of band labels per measurement, and has a keyword argument for determining which averaging strategy to use (mean or median).

In [19]:
import numpy as np


# Defining a simple function
def my_flux_average(flux_array, band_array, method="mean"):
    """Read in an array of fluxes, and return the average of the fluxes by band"""
    res = {}
    for band in np.unique(band_array):
        mask = [band_array == band]  # Create a band by band mask
        band_fluxes = flux_array[tuple(mask)]  # Mask the flux array
        if method == "mean":
            res[band] = np.mean(band_fluxes)
        elif method == "median":
            res[band] = np.median(band_fluxes)
    return res

With the function defined, we next supply it to `Ensemble.batch`. The column labels of the `Ensemble` columns we want to use as arguments must be provided, as well as any keyword arguments. In this case, we pass along `"flux"` and `"band"`, so that the `Ensemble` will map those columns to `flux_array` and `band_array` respectively. We also pass `method='mean'`, which will pass that kwarg along to `my_flux_average`.

In [20]:
# Applying the function to the ensemble
res = ens.batch(my_flux_average, "flux", "band", compute=True, meta=None, method="median")
res

id
0     {'g': 140.03722843377682, 'r': 138.955084796142}
1    {'g': 140.91515408243285, 'r': 141.44229039903...
2    {'g': 139.42093950235392, 'r': 142.21649742828...
3    {'g': 137.01337116218363, 'r': 139.05032340951...
4    {'g': 134.61800608117045, 'r': 139.76505837028...
5    {'g': 135.55144382138587, 'r': 139.41361800167...
6    {'g': 142.93611137557423, 'r': 137.20679606847...
7       {'g': 144.52647796976, 'r': 132.2470836256106}
8     {'g': 144.7469760076462, 'r': 137.5226773361662}
9    {'g': 136.89977482019205, 'r': 136.29794229244...
Name: id, dtype: object

We see that we now have a `Pandas.series` of `my_average_flux` result by object_id (lightcurve). In many cases, this may not be the ideal output for your function. This output is controlled by the `Dask` `meta` parameter. For more information on this parameter, you can read the `Dask` [documentation](https://blog.dask.org/2022/08/09/understanding-meta-keyword-argument). You may pass the `meta` parameter through `Ensemble.batch`, as shown above.

In [21]:
ens.client.close()  # Tear down the ensemble client